---
title: "Informe Técnico: Proyecto Segundo Bimestre - IAyAA"
author: " Colcha. A., Charro.D., Chango. G., Mendieta. M., Moyano. S., Christopher. Z."
lang: es
format:
    pdf:
        toc: true
        toc-depth: 8
execute:
    echo: true
    error: false
    warning: false
code-fold: true
jupyter: python3
---

### Introducción

### Desarrollo

Cargamos las librerías y leemos el dataset completo

In [96]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import seaborn as sns 
import matplotlib.pyplot as plt

In [8]:
data = pd.read_csv('bdd_cp2022.csv', sep = ';', low_memory=False)
data.sample(5)

,INTERVIEW__ID,F1_S1_P0,F1_S1_CPPL,F1_S1_I01,F1_S1_I02,F1_S1_I03,F1_S1_I04,F1_S1_I05,F1_S1_I06,F1_S1_I07,...,F1_S6P02__99,F1_S6P02_19,F1_S6P0301,F1_S6P0302,F1_S6P04,F1_SF_F_LGBTI_DIA,F1_SF_F_LGBTI_MES,F1_SF_LGBTI_ANIO,F1_SF_RES_LEV,F1_SF_MOT_NR
20954,cc6b0043f083437c8486e64244b4497e,14,887,13,1301,130155,913,4,2,0,...,0,,1,2,2,16,9,2022,1,
17329,03d8d7f6cbbf4f859da751c6764bb9a2,10,1801,9,901,90150,670,1,1,0,...,,,2,2,,25,11,2022,1,
7055,3c56124e2a5a4e9b9926a2f853755b5f,3,59,14,1401,140150,8,5,2,0,...,,,1,2,2,9,9,2022,1,
10825,4e6fd14ee16e4fcc81a697a2f90e73c7,7,311,13,1301,130155,913,4,2,0,...,0,,1,2,4,10,12,2022,1,
5177,fe89f19351924642afb4ffa895870a76,6,1131,9,901,90150,670,1,1,0,...,0,,1,2,2,3,12,2022,1,


In [9]:
data.shape

(31321, 398)

### Data preparation
Creamos un array con el nombre de las columnas seleccionadas para el proyecto

In [10]:
features = ['F1_S2_P02', 'F1_S2_P0701','F1_S2_P0702', 'F1_S2_P0703', 'F1_S2_P0704', 
            'F1_S2_P0705', 'F1_S2_P0706', 'F1_S2_P14', 'F1_S2_P15', 'F1_S2_P19', 
            'F1_S2_P25', 'F1_S4P16__2', 'F1_S4P16__3', 'F1_S4P16__4', 'F1_S4P16__5', 
            'F1_S4P16__6','F1_S4P16__7', 'F1_S4P16__8', 'F1_S4P16__9', 'F1_S4P16__10', 
            'F1_S4P16__11','F1_S4P16__12','F1_S4P16__13', 'F1_S4P16__14', 'F1_S4P16__15', 
            'F1_S4P16__16','F1_S4P16__17', 'F1_S4P16__18', 'F1_S4P16__19','F1_S4P16__99', 
            'F1_S5P1901', 'F1_S5P1902', 'F1_S5P1903', 'F1_S5P1904','F1_S5P1905', 
            'F1_S5P1906', 'F1_S5P1907', 'F1_S5P21', 'F1_S5P33', 'F1_S5P3501',
           'F1_S5P3502', 'F1_S5P3503', 'F1_S5P3504', 'F1_S5P3505', 'F1_S5P3506',
            'F1_S5P3601','F1_S5P3602', 'F1_S5P3603', 'F1_S5P42', 'F1_S5P53', 'F1_S5P41__7']

In [107]:
data_select = pd.DataFrame(columns = features)

In [108]:
for i in features:
    data_select[i] = data[i] 

In [109]:
data_select.shape

(31321, 51)

In [110]:
data_select.sample(5)

,F1_S2_P02,F1_S2_P0701,F1_S2_P0702,F1_S2_P0703,F1_S2_P0704,F1_S2_P0705,F1_S2_P0706,F1_S2_P14,F1_S2_P15,F1_S2_P19,...,F1_S5P3503,F1_S5P3504,F1_S5P3505,F1_S5P3506,F1_S5P3601,F1_S5P3602,F1_S5P3603,F1_S5P42,F1_S5P53,F1_S5P41__7
23604,2,1,1,1,1,1,1,2,1,,...,2,2,2,2,1,2,2,1,1,0
16714,1,1,1,1,1,1,1,2,2,,...,1,1,2,1,1,1,1,2,1,0
29753,1,1,1,1,1,1,1,2,2,,...,2,2,2,2,2,2,2,1,1,0
6354,1,1,1,1,1,1,1,2,1,1,...,2,2,2,2,1,1,1,1,1,0
17476,1,1,1,1,1,1,2,2,1,1,...,2,2,2,2,1,1,1,1,2,0


In [111]:
for i in data_select.columns:
    data_select[i] = data_select[i].astype('category')

data_select.dtypes

F1_S2_P02       category
F1_S2_P0701     category
F1_S2_P0702     category
F1_S2_P0703     category
F1_S2_P0704     category
F1_S2_P0705     category
F1_S2_P0706     category
F1_S2_P14       category
F1_S2_P15       category
F1_S2_P19       category
F1_S2_P25       category
F1_S4P16__2     category
F1_S4P16__3     category
F1_S4P16__4     category
F1_S4P16__5     category
F1_S4P16__6     category
F1_S4P16__7     category
F1_S4P16__8     category
F1_S4P16__9     category
F1_S4P16__10    category
F1_S4P16__11    category
F1_S4P16__12    category
F1_S4P16__13    category
F1_S4P16__14    category
F1_S4P16__15    category
F1_S4P16__16    category
F1_S4P16__17    category
F1_S4P16__18    category
F1_S4P16__19    category
F1_S4P16__99    category
F1_S5P1901      category
F1_S5P1902      category
F1_S5P1903      category
F1_S5P1904      category
F1_S5P1905      category
F1_S5P1906      category
F1_S5P1907      category
F1_S5P21        category
F1_S5P33        category
F1_S5P3501      category


In [112]:
for i in data_select.columns:
    print(i, ':', data_select[i].cat.categories)


F1_S2_P02 : Int64Index([1, 2], dtype='int64')
F1_S2_P0701 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0702 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0703 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0704 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0705 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0706 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P14 : Index([' ', '1', '2'], dtype='object')
F1_S2_P15 : Index([' ', '1', '2'], dtype='object')
F1_S2_P19 : Index([' ', '1', '2', '9'], dtype='object')
F1_S2_P25 : Index([' ', '1', '2'], dtype='object')
F1_S4P16__2 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__3 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__4 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__5 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__6 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__7 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__8 : Index([' ', '0', '1'], dtype='object')
F1_S4P16__9 : Index([' ', '0', '1'], dtype='object

In [113]:
data_select = data_select.replace(' ', np.nan)

In [114]:
data_select = data_select.dropna()

In [115]:
#Unificando las variables 'No sabe' y 'No responde' en una sola
data_select = data_select.replace('8', '3')
data_select = data_select.replace('9', '3')

In [116]:
for i in data_select.columns:
    print(i, ':', data_select[i].cat.categories)

F1_S2_P02 : Int64Index([1, 2], dtype='int64')
F1_S2_P0701 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0702 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0703 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0704 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0705 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P0706 : Int64Index([1, 2, 3, 4], dtype='int64')
F1_S2_P14 : Index(['1', '2'], dtype='object')
F1_S2_P15 : Index(['1', '2'], dtype='object')
F1_S2_P19 : Index(['1', '2', '3'], dtype='object')
F1_S2_P25 : Index(['1', '2'], dtype='object')
F1_S4P16__2 : Index(['0', '1'], dtype='object')
F1_S4P16__3 : Index(['0', '1'], dtype='object')
F1_S4P16__4 : Index(['0', '1'], dtype='object')
F1_S4P16__5 : Index(['0', '1'], dtype='object')
F1_S4P16__6 : Index(['0', '1'], dtype='object')
F1_S4P16__7 : Index(['0', '1'], dtype='object')
F1_S4P16__8 : Index(['0', '1'], dtype='object')
F1_S4P16__9 : Index(['0', '1'], dtype='object')
F1_S4P16__10 : Index(['0', '1'], dtype='object')
F1_S4P16

In [117]:
#Group 1


columns = ['F1_S2_P0701','F1_S2_P0702', 'F1_S2_P0703', 
           'F1_S2_P0704', 'F1_S2_P0705', 'F1_S2_P0706']
for i in columns:
    data_select[i] = data_select[i].cat.rename_categories(['0', '1', '2', '3'])
    
#Group 2  
columns = [  'F1_S2_P19','F1_S5P1901', 'F1_S5P1902', 'F1_S5P1903', 'F1_S5P1904','F1_S5P1905', 
            'F1_S5P1906', 'F1_S5P1907', 'F1_S5P21', 'F1_S5P33',  'F1_S5P42', 'F1_S5P53']
for i in columns:
    data_select[i] = data_select[i].cat.rename_categories(['0', '1', '2']) 
    
#Group 3
columns = ['F1_S2_P02','F1_S2_P14', 
           'F1_S2_P15', 'F1_S2_P25','F1_S5P3501',
           'F1_S5P3502', 'F1_S5P3503', 'F1_S5P3504', 'F1_S5P3505', 'F1_S5P3506',
            'F1_S5P3601','F1_S5P3602', 'F1_S5P3603']
for i in columns:
    data_select[i] = data_select[i].cat.rename_categories(['0', '1']) 


In [123]:
for i in data_select.columns:
    print(i, ':', data_select[i].cat.categories)

F1_S2_P02 : Index(['0', '1'], dtype='object')
F1_S2_P0701 : Index(['0', '1', '2', '3'], dtype='object')
F1_S2_P0702 : Index(['0', '1', '2', '3'], dtype='object')
F1_S2_P0703 : Index(['0', '1', '2', '3'], dtype='object')
F1_S2_P0704 : Index(['0', '1', '2', '3'], dtype='object')
F1_S2_P0705 : Index(['0', '1', '2', '3'], dtype='object')
F1_S2_P0706 : Index(['0', '1', '2', '3'], dtype='object')
F1_S2_P14 : Index(['0', '1'], dtype='object')
F1_S2_P15 : Index(['0', '1'], dtype='object')
F1_S2_P19 : Index(['0', '1', '2'], dtype='object')
F1_S2_P25 : Index(['0', '1'], dtype='object')
F1_S4P16__2 : Index(['0', '1'], dtype='object')
F1_S4P16__3 : Index(['0', '1'], dtype='object')
F1_S4P16__4 : Index(['0', '1'], dtype='object')
F1_S4P16__5 : Index(['0', '1'], dtype='object')
F1_S4P16__6 : Index(['0', '1'], dtype='object')
F1_S4P16__7 : Index(['0', '1'], dtype='object')
F1_S4P16__8 : Index(['0', '1'], dtype='object')
F1_S4P16__9 : Index(['0', '1'], dtype='object')
F1_S4P16__10 : Index(['0', '1'], 

Ahora que todas nuestras variables estan estandarizadas como variables categóricas respectivamente, debemos preparar la variable target. Dado que en los datos esta corresponde a `No le gustaría participar en ningún eje de tratamiento` por tanto el valor de `1` nos dice que no va a participar de ningún eje y viceversa. 

Es por eso que, para facilitar la lectura y comprensión de la predicción lo que haremos es invertir estos valores para que la predicción nos responda claramente si `Participaría en algún eje de tratamiento`

In [124]:
data_select['F1_S5P41__7'].value_counts()

1    8769
0     439
Name: F1_S5P41__7, dtype: int64

In [125]:
data_select['F1_S5P41__7'] = data_select['F1_S5P41__7'].replace('0', 'Si')
data_select['F1_S5P41__7'] = data_select['F1_S5P41__7'].replace('1', 'No')

In [126]:
data_select['F1_S5P41__7'] = data_select['F1_S5P41__7'].replace('Si','1')
data_select['F1_S5P41__7'] = data_select['F1_S5P41__7'].replace( 'No', '0')

In [127]:
data_select['F1_S5P41__7'].value_counts()

0    8769
1     439
Name: F1_S5P41__7, dtype: int64

Ahora tenemos claro que `0 = No` y `1 = Si`.


### Feature Selection

Ahora que los datos están listos, separamos la variable target de las features para analizar que la importancia de los datos con los que vamos a trabajar 

In [128]:
y = data_select['F1_S5P41__7']
x = data_select.drop(columns='F1_S5P41__7')

In [129]:
bestfeatures = SelectKBest(score_func=chi2, k=25)
fit = bestfeatures.fit(x,y)
df_scores = pd.DataFrame(fit.scores_)  
df_columns = pd.DataFrame(x.columns)   #nombres de las features
scores = pd.concat([df_columns,df_scores],axis=1)
scores.columns = ['caracteristicas','score']

In [130]:
smallest = scores.nsmallest(25,'score')
smallest

,caracteristicas,score
7,F1_S2_P14,0.004293
23,F1_S4P16__14,0.005197
0,F1_S2_P02,0.012202
13,F1_S4P16__4,0.048196
34,F1_S5P1905,0.051424
36,F1_S5P1907,0.068020
31,F1_S5P1902,0.091709
32,F1_S5P1903,0.095774
30,F1_S5P1901,0.095774
24,F1_S4P16__15,0.101564


Aquí podemos ver 25 de nuestras features que mantiene muy poca relación con nuestra variable target, por tanto vamos a descartar a todas ellas que mantienen una relación menor a `1`.

In [131]:
smallest.set_index(['caracteristicas'], inplace=True)

In [132]:
no_relevant = []
for i in smallest.score.index:
    if smallest.loc[i, :].item() < 1:
        no_relevant.append(i)

In [133]:
no_relevant

['F1_S2_P14',
 'F1_S4P16__14',
 'F1_S2_P02',
 'F1_S4P16__4',
 'F1_S5P1905',
 'F1_S5P1907',
 'F1_S5P1902',
 'F1_S5P1903',
 'F1_S5P1901',
 'F1_S4P16__15',
 'F1_S5P1904',
 'F1_S4P16__11',
 'F1_S4P16__8',
 'F1_S4P16__2',
 'F1_S4P16__10',
 'F1_S5P1906',
 'F1_S4P16__18',
 'F1_S4P16__13',
 'F1_S5P3505',
 'F1_S4P16__16',
 'F1_S4P16__7',
 'F1_S5P33']

In [134]:
data_select = data_select.drop(columns= no_relevant)

In [135]:
data_select.shape

(9208, 29)

In [136]:
y = data_select['F1_S5P41__7']
x = data_select.drop(columns='F1_S5P41__7')

In [137]:
x.columns

Index(['F1_S2_P0701', 'F1_S2_P0702', 'F1_S2_P0703', 'F1_S2_P0704',
       'F1_S2_P0705', 'F1_S2_P0706', 'F1_S2_P15', 'F1_S2_P19', 'F1_S2_P25',
       'F1_S4P16__3', 'F1_S4P16__5', 'F1_S4P16__6', 'F1_S4P16__9',
       'F1_S4P16__12', 'F1_S4P16__17', 'F1_S4P16__19', 'F1_S4P16__99',
       'F1_S5P21', 'F1_S5P3501', 'F1_S5P3502', 'F1_S5P3503', 'F1_S5P3504',
       'F1_S5P3506', 'F1_S5P3601', 'F1_S5P3602', 'F1_S5P3603', 'F1_S5P42',
       'F1_S5P53'],
      dtype='object')

### Data Segregation 

In [ ]:
## lo que separas el training y test

### Model Training

### Model Evaluation

### Conclusiones

### Referencias

1. Apuntes tomados de clase
2. [Pandas](https://pandas.pydata.org/docs/index.html)
2. [Censo Penitenciario Ecuador 2022](https://www.ecuadorencifras.gob.ec/censo-penitenciario-2022/)
